# Hysteresis loop simulation

In [1]:
import mammos_entity as me
import mammos_units as u
import pandas as pd
from mammos_mumag.materials import Materials
from mammos_mumag.mesh import CUBE_20_nm
from mammos_mumag.parameters import Parameters
from mammos_mumag.simulation import Simulation

## Inputs

### Geometry and Mesh

We load one of the meshes coming with `mammos_mumag`, `CUBE_20_nm`, defining a 20 nanometer-side cube. The cube is surrounded by a shell and non-magnetic material in between the material cube and the shell. This is done is order to calculate demagnetisation field which decays to zero at infinity. For details: Imhoff, J. F., et al. "An original solution for unbounded electromagnetic 2D-and 3D-problems throughout the finite element method." IEEE Transactions on Magnetics 26.5 (1990): 1659-1661. 

### Material parameters

We define the material magnetic parameters separately for the 3 domains: the magnetic material, the non-magnetic material and the shell. Therefore the parameters strictly related to the magnetic material will be only defined on the first domain. The other domains, will have magnetic properties equal to zero.

In [3]:
mat = Materials(
    domains=[
        {  # cube
            "theta": 0.0,
            "phi": 0.0,
            "K1": me.Ku(2805163.0590566983, unit=u.J / u.m**3),
            "K2": me.Ku(0.0, unit=u.J / u.m**3),
            "Ms": me.Ms(1189556.9164459957, unit=u.A / u.m),
            "A": me.A(6.015043651663635e-12, unit=u.J / u.m),
        },
        {  # sphere
            "theta": 0.0,
            "phi": 0.0,
            "K1": me.Ku(0.0, unit=u.J / u.m**3),
            "K2": me.Ku(0.0, unit=u.J / u.m**3),
            "Ms": me.Ms(0.0, unit=u.A / u.m),
            "A": me.A(0.0, unit=u.J / u.m),
        },
        {  # shell
            "theta": 0.0,
            "phi": 0.0,
            "K1": me.Ku(0.0, unit=u.J / u.m**3),
            "K2": me.Ku(0.0, unit=u.J / u.m**3),
            "Ms": me.Ms(0.0, unit=u.A / u.m),
            "A": me.A(0.0, unit=u.J / u.m),
        },
    ],
)

### Simulation parameters

We finally define all the simulation parameters we will use. For an exhaustive explanation of these parameters, check [the documentation](https://mammos-project.github.io/mammos/api/mammos_mumag.parameters.html#module-mammos_mumag.parameters).

In [4]:
par = Parameters(
    size=1.0e-9,
    scale=0,
    m_vect=[0, 0, 1],
    hstart=10,
    hfinal=-10,
    hstep=-2,
    h_vect=[0.01745, 0, 0.99984],
    mstep=0.4,
    mfinal=-1.2,
    tol_fun=1e-10,
    tol_hmag_factor=1,
    precond_iter=10,
)

## Putting all inputs in a Simulation object

To define a `Simulation` object, we need to define a mesh, a `Materials` object, and a `Parameters` object as shown above.

In [5]:
sim = Simulation(
    mesh_filepath=CUBE_20_nm,
    materials=mat,
    parameters=par,
)

Note that all of this could also have been defined using file paths:
```python
sim = Simulation(
    mesh_filepath="...",
    materials_filepath="...",
    parameters_filepath="...",
)
```

## Hysteresis loop calculation

To compute the demagnetization curve, we use `sim.run_loop` method. Here, we can specify the optional argument `outdir` and `name`. While the first identifies the output directory where the input and output files will be stored (also where the script is executed), the `name` argument defines the names of the output files.

The `outdir` argument can be used to give an ID to the simulation.

In [6]:
sim.run_loop(outdir="out/loop", name="cube")

The `out/loop` directory looks like:
```console
$> tree out/loop/
out/loop/
├── cube_0001.vtu
├── cube_0002.vtu
├── cube.dat
├── cube.fly
├── cube.krn
├── cube.p2
├── cube_stats.txt
└── info.json
```

The file `cube.dat` has the following information:

In [7]:
pd.read_csv("out/loop/cube.dat", delimiter=" ", names=["idx", "mu0_Hext", "polarisation", "E"])

,idx,mu0_Hext,polarisation,E
0,1,10.0,1.494715,-1.442296e+07
1,1,8.0,1.494674,-1.204407e+07
2,1,6.0,1.494610,-9.665272e+06
3,1,4.0,1.494501,-7.286604e+06
4,1,2.0,1.494289,-4.908181e+06
5,1,0.0,1.493791,-2.530279e+06
6,1,-2.0,1.492077,-1.538731e+05
7,1,-4.0,1.463528,2.211992e+06
8,2,-6.0,-1.494610,-9.665272e+06


- `idx`: the id of the magnetisation vector field appended to the `vtu` file name. The `vtu` files contain the magnetisation corresponding to the row where the given value of `idx` was first observed.
- `mu0 Hext`: the value of $\mu_0 H_{\mathsf{ext}}$ (in T) where $\mu_0$ is the permability of vacuum and $H_{\mathsf{ext}}$ is the value of the external field.
- `pol`: the componenent of magnetic polarisation (in T) parallel to the direction of the external field.
- `E`: the energy density (in J/m$^3$) of the current state.

Further execution statistics are found in file `cube_stats.txt`

In [8]:
with open("out/loop/cube_stats.txt") as file:
    print(file.read())

Memory before escript2jax: 194.82421875 MB.
Memory after  escript2jax: 424.796875 MB.
Memory after garbage collection: 424.796875 MB.


> **NOTE:** It is important to note that there is a convenience function [mammos_mumag.hysteresis.run](https://mammos-project.github.io/mammos/examples/mammos-mumag/quickstart.html#mammos_mumag-quickstart) that does not require the user to input detailed material and simulation parameters to run hysteresis simulations.